# COMS 4995_002 Deep Learning Assignment 1
Due on Thursday, Feb 8, 11:59pm

This assignment can be done in groups of at most 2 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Samuel Cohen, slc2206

Member 2: Jason Zhao, jsz2107

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [63]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.parameters = {}
        self.num_layers = len(layer_dimensions)
        self.drop_prob = 0
        self.reg_lambda = 0
        
        #Init parameters (Key:)
        self.parameters["weights"] = {}
        self.parameters["biases"] = {}
        
        #Should be using xavier random init!!!

        #Loop over every layer besides the last
        for i in range(self.num_layers-1):
            #Set weight matrix of layer i to layer i+1
            self.parameters["weights"][i] = 0.01*np.random.randn(layer_dimensions[i+1], layer_dimensions[i])
            #Set bias vector biases from layer i to layer i+1 (bias vector length = num neurons in layer i+!)
            self.parameters["biases"][i] = 0.01*np.random.randn(layer_dimensions[i+1]).reshape(layer_dimensions[i+1],1)


    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        #print("IN AFFINE FORWARD")
        #'A' is a matrix holding the activations in a given layer for each example.
        
        #Absorb the bias vector into the weight matrix 
        #print("input.shape:")
        #print(A.shape)
        
        #print("W.shape:")
        #print(W.shape)
        
        #print("b.shape:")
        #print(b.shape)
        W_mod = np.concatenate((W, b), axis=1)

        A_mod = np.append(A, np.matrix(np.ones(A.shape[1])), axis = 0)
        affine_prod = np.matmul(W_mod, A_mod)
        
        #Store to be used in backprop?
        #cache = [A, W, b, affine_prod]
        cache = {}
        cache['A'] = A
        cache['W'] = W
        cache['b'] = b
        cache['affine_prod'] = affine_prod
        
        return affine_prod, cache


    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        
        # We're only using ReLU in this network
        return self.relu(A)


    def relu(self, X):
        """ The ReLU function to calculate activations."""
        A = np.maximum(0, X)
        return A

            
    def dropout(self, A, prob):
        """
        :param A: 
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        pass
        return A, M

    
    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        #print("In forward prop!!!!")
        cache = {}
        #print("trype cache: " + str(type(cache)))
#         cache["affine"] = {}
#         cache["activations"] = {}
        
        previous_activations = X

        for i in range(self.num_layers - 1):
            
            #print("self.parameters[weights]" + str(self.parameters["weights"].keys()))
            #print("self.parameters[biases]" + str(self.parameters["biases"].keys()))
            #print("i=" +str(i))
            affine_prod, cache_dic = self.affineForward(previous_activations, self.parameters["weights"][i], self.parameters["biases"][i])
            cache[i] = cache_dic # We dont know what affine_layer is returning as of yet !!!!!!
            activations = self.activationForward(affine_prod)
            cache[i]["activations"] = activations
            
            previous_activations = activations
            
        return activations, cache
    
    
    def softmax(self, zL):
        """
        The softmax is used for multi-class classification and computes a probability 
        in (0, 1) for each class c = 1...C, which sums to 1.
        """
        #zL: pre-activation in the last layer (L)
        return np.exp(zL) / np.sum(np.exp(zL), axis = 0)
    
    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        
        """
        * Note: 
            We want to maximize the log likelihood of our computed probability distribution matching
            the true distribution, in the case of a "loss" function we want to minimize the negative log-likelihood
            to achieve this same result.
        
        * Loss is defined:
            sum up the computed probabilities corresponding to the correct label. If these are high values it means
            that we were "close" to being correct and take the negative log of this sum
        
        *Imagine we had probability mass = 0 on the correct label. Then log of 0 is -infinity, 
            so negative log of 0 is +infinity (i.e HUGE loss if you totally mispredict the correct label.)
        
        """
        #Get the number of training examples
        m = y.shape[0] 
        
        #Convert last-layer activations into probabilities summing to 1
        AL_softmax = self.softmax(AL)       
        
        #Create a vector that only holds the computed probabilities corresponding to correct labels
        correct_label_probs = AL_softmax[y, range(m)]
        
        
        #Why do we have to divide by m? Like we do we even need an "average" of loss as opposed to a overall loss?
        #Compute cost by taking the average of the negative log of the correct label probabilities?
        cost = - np.sum(np.log(correct_label_probs)) / m 
                
#         #Build a mask to pull out only correct label's calculated probabilities
#         num_classes = len(AL)
#         mask = np.zeros(num_classes, m)
#         mask[y, np.arange(m)] = 1
#         #Element-wise multiply the mask and AL_softmax to only retain the correct label probabilities
#         correct_probs = np.multiply(AL_softmax, mask)
#         #Convert to a vector of correct label calculated probabilities
        #prediction_differences = np.sum(np.abs(y - AL_softmax), axis = 1) / m #This averages across all examples 
        #cost = - np.sum(np.log(prediction_differences)) / m
        
        
        if self.reg_lambda > 0:
            # add regularization
            pass
       
        
        #Find the gradient of cost
        #Differenatiate w.r.t the only correct label. So subtract the 1 correct label 1
        # from the correct label in the predictions look at stanford
        #Output of softmax layer - 1hot encoding
        
        #Create a 1-hot encoded matrix where the row corresponds the class and the column corresponds to the example
        num_classes = layer_dimensions[-1]

        #print(type(m))
        #print(type(num_classes))
        one_hot = np.zeros((num_classes, m))
        #print(one_hot.size)
        one_hot[y, range(m)] = 1
        
        # TODO: Should we be dividing by m?
        dAL = (AL_softmax - one_hot) / m
        return cost, dAL 

    
    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        
        A = cache['A']
        W = cache['W']
        b = cache['b']
        affine_prod = cache['affine_prod']

        dW = np.dot(A, dA_prev.T)
        print(dA_prev.shape)
        db = np.sum(dA_prev, axis=0)
        dA = self.relu_derivative(np.dot(dA_prev.T, W), )
        
        #Calculate the gradient on the 
       
        #FROM PIAZZA: The forward pass is WT*A + b. When you differentiate w.r.t. A, only WT remains, So bias won't be incorporated.


        
        #dA_prev is the value for layer l+1 since you are doing backpropagation
        
        
        
        return dA, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        #if derivative 0 then return 0, else Return same thing (b/c multiplying by 1)
        
        #The functionality of this function can also just be absorbed into the backpropagation function, sort of excessive
        #Have to turn of the neurons that were 0, when to drop the terms if it wasnt activated
        return self.relu_derivative(dA, cache[0])
        
        
    def relu_derivative(self, dx, cached_x):
        """
        cached_x: num_neurons X num examples
        
        The ReLU activation function turns the input off (to zero) when the input is negative.
        So when we are backpropagating, we need to know which gradients should be turned off. 
        (This is the reason for two arguments to this function)        
        """
        
        mask = cached_x > 0
        
        relu_deriv = np.multiply(dx, mask.T)
        
        return relu_deriv
        #return dx

        
    def dropout_backward(self, dA, cache):

        return dA

    
    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        
        #First get the gradient w.r.t the last layer
        #cost, dAL = costFunction(AL, Y)
        
        #This cache is the cache returned in affineForward, but the cache in the arg is the one from forward prop???!
        #dA, dW, db = affineBackward(self, dA_prev, cache)
        
        gradients = []
        
        # Go through layers backwards starting at the index of the last layer (num_layers - 1)
        # minus 1 because there are num_layers - 1 weight matricies
        for i in range(self.num_layers - 2, -1, -1):
            
            #cache[i]['prev_activations'] = cache[i - 1]['activations']
            gradients.append(self.affineBackward(dAL, cache[i]))
            
            if self.drop_prob > 0:
                #call dropout_backward
                pass      
            
        if self.reg_lambda > 0:
            # add gradients from L2 regularization to each dW
            pass
        
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        
        for i in range(self.num_layers):
            self.parameters["weights"][i] -= alpha * gradients[0]
            self.parameters["biases"][i] -= alpha * gradients[1]

            
    def shuffle(self, X, y):
        size = len(X)
        for i in range(10 * size):
            r = np.random.randint(0, size)
            tempX = X[0]
            tempy = y[0]
            X[0] = X[r]
            y[0] = y[r]
            X[r] = tempX
            y[r] = tempy
        
        
    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        
        #X.shape: 3072 X 50000 (i.e each training feature is a row and each example is a column)
        # Each image is 32X32 pixels = 32*32 = 1024 * 3 colors = 3072 and 50000 images total
        
        
        #Can get minibatch one of two ways, we choose to sample w/o replacement. Try with later.
        print("Shuffling...")
        self.shuffle(X, y)
        print("Done.")
        
        #print("official shape of X: " + str(X.shape))
        for i in range(0, iters):
            # get minibatch
            train_x, train_y = self.get_batch(X, y, batch_size, i)
            #print("train_x shape and train_y shape!!!!")
            #print(train_x.shape)
            #print(train_y.shape)
            
            #print("y.shape[0] must equal X.shape[1] == "+ str(y.shape[0]) + ", "+str(X.shape[1]))
            
            # forward prop
            AL, cache = self.forwardPropagation(train_x)
            
            # compute loss
            loss, dAL = self.costFunction(AL, train_y)

            # compute gradients
            grads = self.backPropagation(dAL, train_y, cache)

            # update weights and biases based on gradient
            self.updateParameters(grads, alpha)

            if i % print_every == 0:
                # print cost, train and validation set accuracies
                print("iter=" + str(i) + "   loss=" + str(loss))
                
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        AL, cache = self.forwardPropagation(X)
        
        #Create vector holding index (class prediction) corresponding to the highest value in each column for each example
#         y_pred = AL.argmax(axis = 0)


        return AL.argmax(axis = 0)#y_pred

    
    def get_batch(self, X, y, batch_size, current_iter):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        #What happens if batch_size and iterations dont perfectly align with number of samples? The last batch could be very small?? 
        
        X_batch = X[:,current_iter*batch_size : current_iter*batch_size+batch_size]
        y_batch = y[current_iter*batch_size : current_iter*batch_size+batch_size]
        
        #print("in get_batch")
        #print("xbatch shape: " + str(X_batch.shape))
        #print("current_iter*batch_size = "+str(current_iter*batch_size))
        #print("current_iter*batch_size+batch_size = " +str(current_iter*batch_size+batch_size))
        
        
        return X_batch, y_batch

In [44]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [10]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [11]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [62]:
layer_dimensions = [X_train.shape[0], 40, 10]  # including the input and output layers
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train)

Shuffling...
Done.
in get_batch
xbatch shape: (3072, 100)
current_iter*batch_size = 0
current_iter*batch_size+batch_size = 100


KeyError: 'prev_activations'

In [7]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

input.shape:
(3072, 10000)
W.shape:
(40, 3072)
b.shape:
(40,)


ValueError: all the input arrays must have same number of dimensions

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Improving the performance

In [ ]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0, reg_lambda=0)
NN2.train(X_train, y_train, iters=1000, alpha=0.00001, batch_size=1000, print_every=10)

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')

Write down results for Part 2 here:
...